## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[complaints_st_partner]]

[[cameras_st_partner]]

[[intercoms_st_partner]]

[[installation_point_st_partner]]

[[entries_installation_points_dir_partner]]

[[companies_dir_partner]]

[[t_all_installetion_points]]
___

In [3]:
query_text = """--sql
WITH data_range AS(
SELECT
	report_date
FROM db1.complaints_st_partner_ch
),
--
cameras_st_partner AS(
SELECT
	report_date,
	 camera_uuid,
	 installation_point_id
FROM db1.cameras_st_partner_ch
WHERE report_date in data_range
    ),
--
intercoms_st_partner AS(
SELECT
	report_date,
	 intercom_uuid,
	 installation_point_id
FROM db1.intercoms_st_partner_ch
WHERE report_date in data_range
    ),
companies AS(
SELECT
	report_date,
	parent_uuid,
	ins_p_s_p.partner_uuid AS partner_uuid,
	partner_lk,
	tin,
	company_name 
FROM db1.installation_point_st_partner_ch AS ins_p_s_p
LEFT JOIN db1.entries_installation_points_dir_partner_ch AS eipdp 
	ON eipdp.installation_point_id = ins_p_s_p.installation_point_id
LEFT JOIN db1.companies_dir_partner_ch AS cdp
	ON cdp.partner_uuid = ins_p_s_p.partner_uuid
WHERE report_date in data_range
	)	
--
SELECT
	 com_s_p.report_date AS report_date,
    `issue_id`, 
    `src_type`, 
    `src_uuid`, 
    `src_address_uuid`, 
    `src_address_type`, 
    `device_uuid`, 
    `device_type`, 
    `err_code`, 
    `err_src`,
    `err_func`, 
    `aasm_state`,
    `process_error`, 
    `actor_type`, 
    `actor_identifier`,
    com_s_p.`created_at` AS created_at,
    `updated_at`,
    com_s_p.`installation_point_id` AS installation_point_id,
    `city`,
    `full_address`,
    `region`,
    all_in_p.`parent_uuid` AS parent_uuid,
    `partner_lk`,
	`tin`,
	`company_name` 
FROM    
	(SELECT
	    com_s_p.`report_date` AS report_date,
	    `issue_id`, -- это на будущее,  когда будет группировка жалоб по ишьб
	    `src_type`, --какая сущность пожаловалсь (Citizen/Camera/Intercom)
	    `src_uuid`, -- uuid этой сущности
	    `src_address_uuid`, -- адрес этой сущности
	    `src_address_type`, -- адрес этой сущности
	    `device_uuid`, -- uuid устройтсва на которое жалуется сущность 
	    `device_type`, 
	    `err_code`, --  код ошибки intercom_offline / video / no_intercom
	    `err_src`,	-- откуда пришло MOB (мобилка) / WEB (веб интерфейсы для партнеров) / API (бэк )/ DEV (устройство)
	    `err_func`, --sync (там еще будет)
	    `aasm_state`,	-- статус жалобы pending / processed / failed  
	    `process_error`, -- ошибка если статус failed  
	    `actor_type`, -- какой живой человек с сердцем пожаловался или авто система (будет либо Citizen, Partner, auto)
	    `actor_identifier`, --uuid / id актэра
	    `created_at`,
	    `updated_at`,
	      if(cameras_st_partner.`installation_point_id` = 0 
	      OR cameras_st_partner.`installation_point_id` IS NULL, 
	      intercoms_st_partner.`installation_point_id`,  cameras_st_partner.`installation_point_id`) AS installation_point_id
	FROM db1.complaints_st_partner_ch AS com_s_p
	LEFT JOIN cameras_st_partner 
		ON cameras_st_partner.report_date = com_s_p.report_date 
		AND cameras_st_partner.camera_uuid = com_s_p.device_uuid
	LEFT JOIN intercoms_st_partner 
		ON intercoms_st_partner.report_date = com_s_p.report_date 
		AND intercoms_st_partner.intercom_uuid = com_s_p.device_uuid
		) AS com_s_p
	LEFT JOIN db1.t_all_installetion_points AS all_in_p
		ON all_in_p.report_date = com_s_p.report_date
		AND all_in_p.installation_point_id = com_s_p.installation_point_id
	LEFT JOIN companies 
		ON companies.report_date = all_in_p.report_date
		AND companies.parent_uuid = all_in_p.parent_uuid
"""

ch.get_schema(query_text)

(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code` String,
    `err_src` String,
    `err_func` String,
    `aasm_state` String,
    `process_error` String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64,
    `updated_at` DateTime64,
    `installation_point_id` Int64,
    `city` String,
    `full_address` String,
    `region` String,
    `parent_uuid` String,
    `partner_lk` String,
    `tin` String,
    `company_name` String
)


In [4]:
query_text = """--sql
CREATE TABLE db1.t_complaints_st_partner
(
    `report_date` Date,
    `issue_id` String,
    `src_type` String,
    `src_uuid` String,
    `src_address_uuid` String,
    `src_address_type` String,
    `device_uuid` String,
    `device_type` String,
    `err_code` String,
    `err_src` String,
    `err_func` String,
    `aasm_state` String,
    `process_error` String,
    `actor_type` String,
    `actor_identifier` String,
    `created_at` DateTime64,
    `updated_at` DateTime64,
    `installation_point_id` Int64,
    `city` String,
    `full_address` String,
    `region` String,
    `parent_uuid` String,
    `partner_lk` String,
    `tin` String,
    `company_name` String
)
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [5]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_complaints_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 37 MINUTE TO db1.t_complaints_st_partner AS
WITH data_range AS(
SELECT
	report_date
FROM db1.complaints_st_partner_ch
),
--
cameras_st_partner AS(
SELECT
	report_date,
	 camera_uuid,
	 installation_point_id
FROM db1.cameras_st_partner_ch
WHERE report_date in data_range
    ),
--
intercoms_st_partner AS(
SELECT
	report_date,
	 intercom_uuid,
	 installation_point_id
FROM db1.intercoms_st_partner_ch
WHERE report_date in data_range
    ),
companies AS(
SELECT
	report_date,
	parent_uuid,
	ins_p_s_p.partner_uuid AS partner_uuid,
	partner_lk,
	tin,
	company_name 
FROM db1.installation_point_st_partner_ch AS ins_p_s_p
LEFT JOIN db1.entries_installation_points_dir_partner_ch AS eipdp 
	ON eipdp.installation_point_id = ins_p_s_p.installation_point_id
LEFT JOIN db1.companies_dir_partner_ch AS cdp
	ON cdp.partner_uuid = ins_p_s_p.partner_uuid
WHERE report_date in data_range
	)	
--
SELECT
	 com_s_p.report_date AS report_date,
    `issue_id`, 
    `src_type`, 
    `src_uuid`, 
    `src_address_uuid`, 
    `src_address_type`, 
    `device_uuid`, 
    `device_type`, 
    `err_code`, 
    `err_src`,
    `err_func`, 
    `aasm_state`,
    `process_error`, 
    `actor_type`, 
    `actor_identifier`,
    com_s_p.`created_at` AS created_at,
    `updated_at`,
    com_s_p.`installation_point_id` AS installation_point_id,
    `city`,
    `full_address`,
    `region`,
    all_in_p.`parent_uuid` AS parent_uuid,
    `partner_lk`,
	`tin`,
	`company_name` 
FROM    
	(SELECT
	    com_s_p.`report_date` AS report_date,
	    `issue_id`, -- это на будущее,  когда будет группировка жалоб по ишьб
	    `src_type`, --какая сущность пожаловалсь (Citizen/Camera/Intercom)
	    `src_uuid`, -- uuid этой сущности
	    `src_address_uuid`, -- адрес этой сущности
	    `src_address_type`, -- адрес этой сущности
	    `device_uuid`, -- uuid устройтсва на которое жалуется сущность 
	    `device_type`, 
	    `err_code`, --  код ошибки intercom_offline / video / no_intercom
	    `err_src`,	-- откуда пришло MOB (мобилка) / WEB (веб интерфейсы для партнеров) / API (бэк )/ DEV (устройство)
	    `err_func`, --sync (там еще будет)
	    `aasm_state`,	-- статус жалобы pending / processed / failed  
	    `process_error`, -- ошибка если статус failed  
	    `actor_type`, -- какой живой человек с сердцем пожаловался или авто система (будет либо Citizen, Partner, auto)
	    `actor_identifier`, --uuid / id актэра
	    `created_at`,
	    `updated_at`,
	      if(cameras_st_partner.`installation_point_id` = 0 
	      OR cameras_st_partner.`installation_point_id` IS NULL, 
	      intercoms_st_partner.`installation_point_id`,  cameras_st_partner.`installation_point_id`) AS installation_point_id
	FROM db1.complaints_st_partner_ch AS com_s_p
	LEFT JOIN cameras_st_partner 
		ON cameras_st_partner.report_date = com_s_p.report_date 
		AND cameras_st_partner.camera_uuid = com_s_p.device_uuid
	LEFT JOIN intercoms_st_partner 
		ON intercoms_st_partner.report_date = com_s_p.report_date 
		AND intercoms_st_partner.intercom_uuid = com_s_p.device_uuid
		) AS com_s_p
	LEFT JOIN db1.t_all_installetion_points AS all_in_p
		ON all_in_p.report_date = com_s_p.report_date
		AND all_in_p.installation_point_id = com_s_p.installation_point_id
	LEFT JOIN companies 
		ON companies.report_date = all_in_p.report_date
		AND companies.parent_uuid = all_in_p.parent_uuid
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [9]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_complaints_st_partner
    ORDER BY report_date DESC
    limit 10
    """

ch.query_run(query_text)


report_date,issue_id,src_type,src_uuid,src_address_uuid,src_address_type,device_uuid,device_type,err_code,err_src,err_func,aasm_state,process_error,actor_type,actor_identifier,created_at,updated_at,installation_point_id,city,full_address,region,parent_uuid,partner_lk,tin,company_name
date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[ns],datetime[ns],i64,str,str,str,str,str,str,str
2025-08-18,"""""","""citizen""","""fffe3d4f-f9de-45f4-aa84-369bb5…","""2e3e3155-99b9-4499-b9d8-34f04b…","""entry""","""41a7be03-5308-48a8-8bef-837054…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 05:47:42,2025-08-18 05:47:43,118192,"""Актобе""","""Казахстан, Актюбинская область…","""Актюбинская область""","""53aa1881-d852-4c7c-b1e9-77f945…","""120377""","""""","""ТОО ЦИФРАЛ ЦТО"""
2025-08-18,"""""","""citizen""","""fffe3d4f-f9de-45f4-aa84-369bb5…","""2e3e3155-99b9-4499-b9d8-34f04b…","""entry""","""41a7be03-5308-48a8-8bef-837054…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 05:47:42,2025-08-18 05:47:43,118192,"""Актобе""","""Казахстан, Актюбинская область…","""Актюбинская область""","""53aa1881-d852-4c7c-b1e9-77f945…","""120377""","""""","""ТОО ЦИФРАЛ ЦТО"""
2025-08-18,"""""","""citizen""","""fffe3d4f-f9de-45f4-aa84-369bb5…","""2e3e3155-99b9-4499-b9d8-34f04b…","""entry""","""41a7be03-5308-48a8-8bef-837054…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 05:47:42,2025-08-18 05:47:43,118192,"""Актобе""","""Казахстан, Актюбинская область…","""Актюбинская область""","""53aa1881-d852-4c7c-b1e9-77f945…","""120377""","""""","""ТОО ЦИФРАЛ ЦТО"""
2025-08-18,"""""","""citizen""","""fffe3d4f-f9de-45f4-aa84-369bb5…","""2e3e3155-99b9-4499-b9d8-34f04b…","""entry""","""41a7be03-5308-48a8-8bef-837054…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 05:47:42,2025-08-18 05:47:43,118192,"""Актобе""","""Казахстан, Актюбинская область…","""Актюбинская область""","""53aa1881-d852-4c7c-b1e9-77f945…","""120377""","""""","""ТОО ЦИФРАЛ ЦТО"""
2025-08-18,"""""","""citizen""","""fffe3d4f-f9de-45f4-aa84-369bb5…","""2e3e3155-99b9-4499-b9d8-34f04b…","""entry""","""41a7be03-5308-48a8-8bef-837054…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 05:47:42,2025-08-18 05:47:43,118192,"""Актобе""","""Казахстан, Актюбинская область…","""Актюбинская область""","""53aa1881-d852-4c7c-b1e9-77f945…","""120377""","""""","""ТОО ЦИФРАЛ ЦТО"""
2025-08-18,"""""","""citizen""","""fffd6dc4-cefd-4a31-ad4b-d0e0a8…","""ee19d75a-5629-4628-98fd-1e2070…","""entry""","""2fd57153-82d7-40ab-a3e0-76083f…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 01:05:12,2025-08-18 01:05:12,611298,"""Хабаровск""","""Россия, Хабаровский край, горо…","""Хабаровский край""","""f7214b4e-3ee2-4df1-aa25-35cfbf…","""135806""","""2723127644""","""ООО Одеон М ПРО"""
2025-08-18,"""""","""citizen""","""fff56755-fe95-4490-849a-a3c777…","""9dea3370-97da-4bfb-824d-5bb7ec…","""entry""","""4d6ee29d-5170-4ec8-a93f-8effd5…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 15:54:02,2025-08-18 15:54:03,109520,"""Калининград""","""Россия, Калининградская област…","""Калининградская область""","""a3462024-cc57-40ca-b547-093118…","""120345""","""""","""Цифрал Сервис УК"""
2025-08-18,"""""","""citizen""","""fff54046-b253-4e05-999c-122daa…","""15d5fee0-b2a9-44d3-a8f8-649e8d…","""entry""","""c8bc373f-bb04-4e3b-afbd-579789…","""camera""","""video""","""MOB""","""""","""processed""","""""","""auto""","""""",2025-08-18 08:23:34,2025-08-18 08:23:35,761257,"""Курган""","""Россия, Курганская область, го…","""Курганская область""","""51217a94-6061-4205-980b-274872…","""132902""","""667219732450""","""ИП Коровин А.В. ПРО"""
2025-08-18,"""""","""citizen""","""fff4bdab-a83c-4977-8acd-255b39…","""0c2d020d-b502-4

### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_complaints_st_partner DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [3]:
query_text = """--sql
    DROP TABLE db1.t_complaints_st_partner_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [3]:
query_text = """--sql
    DROP TABLE db1.t_complaints_st_partner
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [16]:
query_text = """
SYSTEM REFRESH VIEW db1.t_complaints_st_partner_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
